In [23]:
import os

In [24]:
%pwd

'd:\\recommendation-engine'

In [25]:
os.chdir("../")

In [26]:
%pwd

'd:\\'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    unzip_dir: Path

In [37]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories


In [40]:
class ConfigurationManager:
    def __init__(self):
        config_filepath = CONFIG_FILE_PATH
        params_filepath = PARAMS_FILE_PATH
        schema_filepath = SCHEMA_FILE_PATH

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [45]:
import os
import urllib.request as request
import zipfile
from hybrid_recommender import logger
from hybrid_recommender.utils.common import get_size

ModuleNotFoundError: No module named 'hybrid_recommender'